<a href="https://colab.research.google.com/github/Benned-H/Reading_List/blob/master/Notes/Visual_Navigation_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cognitive Mapping and Planning for Visual Navigation [[pdf]](https://arxiv.org/pdf/1702.03920.pdf)

**Abstract** - This work introduces a neural architecture for *navigation in novel environments.* They map from first-person views to sequences of actions towards goals in an environment. The **Cognitive Mapper and Planner** (CMP) is based on two ideas:
1. Unifying the architecture between mapping and planning so that mapping is driven by the needs of the task.
2. Spatial memory with ability to plan with incomplete observations.

The CMP plans by constructing a top-down belief map of the world and using an ANN planner to choose an action each time step. The agent was tested on simulated navigation problems built from scans of real buildings. CMP outperforms a few other architectures and also extends to semantically specified goals. CMP was also somewhat successfully deployed on real robots.

## 1. Introduction

Humans utilize numerous common-sense rules and heuristics when navigating, almost without thinking. Such insights also need to be learned for robot navigation, and that's our goal here. Earlier work based on SLAM (simultaneous localization and mapping) used LIDAR to build maps purely from observed structures. This uninformed exploration is far from the most useful way for robots to navigate in 2019. More recent approaches address this with end-to-end pixel-to-action mappings, yet even rats build cognitive maps for navigating.

These issues all motivate the CMP approach, which consists of a spatial memory to capture the layout of the world and a path planner that works with partial information. The two are put together to help capture regularities in the world. The mapper takes input views over time and produces a *egocentric multi-scale top-down belief map* of the world. Defining these terms:
* Egocentric - A way of understanding spatial relations that represents locations in space relative to the axes of the agent (left-right, front-back, up-down). This is in contrast to allocentric representations (object-to-object). [Source](http://www.nmr.mgh.harvard.edu/mkozhevnlab/?page_id=308)
* Multi-scale - Refers to maps that display data in different ways across a range of scales. Not sure how this will apply to our robot but we'll see.
* Top-down - The map is produced as a bird's eye view of the environment, as if the viewer of the map is above the world looking down.
* Belief map - Not very clearly defined online, it's probably some way for the agent to represent its current hypothesis/*beliefs* about the environment, even if it's uncertain about parts.

Using this map, the planner outputs the optimal path to take to reach the agent's goal.

*Mapping*: Each time step, the agent updates its map by transforming it to the current coordinate frame and then incorporating new visual information. The entire process is framed as a learning problem (the CNN predicts updates to the map), which means that every step is differentiable. This in turn allows complete end-to-end training, which means the model can adapt for patterns in real-world buildings without explicit supervision.

*Planning*: The agent uses a trainable, differentiable, and hierarchical value iteration. These terms, defined, tell us:
* A **trainable** planner means that we inherently account for partially observed environments, as we learn what and when to explore.
* **Differentiable** is helpful because SGD and so on.
* **Hierarchical** path planning allows a logarithmic time complexity w.r.t. the number of steps to the goal.   

Because their entire architecture learns everything from data, it can learn regularities of indoor environments, it's more robust to mapping errors, and it can be used online in new environments.

## 2. Related Work

Standard approaches to navigation decompose the task into two distinct stages: mapping and planning. This typically allows separate development of these stages, but also prevents the two stages from exploiting the needs of the other. Geometric approaches to mapping have used modalities such as range sensors, RGB cameras, and RGB-D cameras, while learning-based approaches produce generic task-independent maps. A few works also leveraged semantics for more informed navigation.

As an alternative to discrete mapping and planning, RL has been used for robotics tasks. The main challenge here is the processing of complex sensory image inputs, but recent DRL has learned end-to-end pixel-to-actions policies. This work builds on [Tamar et al.](https://papers.nips.cc/paper/6046-value-iteration-networks.pdf) which studied how explicit planning could be added to DRL agents with memory (without the case of first-person visual navigation). Of the many DRL works on navigation, most focus on random synthetic mazes with little structure, and thus learn nothing about the exploitable patterns present in real-world environments.

The most relevant work to this one is that of [Zhu et al.](https://arxiv.org/pdf/1609.05143.pdf) which studied first-person navigation in realistic environments. They proposed a feed-forward model which can be pretrained in one environment and finetuned in another. Theirs couldn't map, plan, or explore, but this paper does and also explicitly handles generalizing to new environments.

**Relation to Contemporary Work**: This work uses scans of real world environments to construct realistic simulations. In the last year, the field has pushed in this direction, with many large-scale scan datasets and simulation environments built from such scans coming out. Goals are also being specified beyond desired locations, but also finding objects of interest. A lot of work has also considered the best format for building representations, whether that be spatially in 2D, as fully-connected LSTMS, or topologically.

## 3. Problem Setup

They conducted their experiments in static simulated real-world environments in order to avoid the challenges of low-level control. This also made it easier to fully evaluate the system. The robot was modelled as a cylinder of fixed radius and height, with vision sensors (RGB or depth sensors) mounted at a fixed height and pitch (vertical rotation). The robot was given low-level controllers with high-level macro-actions $\mathcal{A}_{x,\theta}$:
* $a_0$ - Stay in place
* $a_1$ - Rotate left by $\theta$
* $a_2$ - Rotate right by $\theta$
* $a_3$ - Move forward by $x$ cm

The environment is also assumed to be a grid world, and the robot moves between nodes in this graph. The robot can also access its precise egomotion (the 3D motion of a camera in an environment) and thus knows exactly where its camera is in the world. The authors note that perfect visual odometry is learnable, but joint learning that is left for future work.

The goal is to learn policies for unseen environments, so two navigation tasks are studied: the *geometric* task of moving to a target location in the robot's coordinate frame, and a semantic task of going to some object of interest (e.g. a chair). The problem is formally defined as follows. At given time step $t$, assume the robot is at some global position (in the world coordinate frame) $P_t$. At each $t$ the robot receives as input the image of the environment $\mathcal{E}$, $I_t=I(\mathcal{E},P_t)$ and a target location as either $(x_t^g,y_t^g,\theta_t^g)$ or a semantic goal. The navigation problem is to learn a policy that uses these inputs (current image, egomotion and target specification) to output actions that convey the robot to the goal ASAP.

**Experimental Testbed** - They used the *Stanford large-scale 3D indoor spaces (S3DIS) dataset* (see [here](http://buildingparser.stanford.edu/images/3D_Semantic_Parsing.pdf)), which consists of 3D scans of 6 large-scale indoor areas in 3 buildings of educational and office use. It was collected using the Matterport scanner: a brand of camera that creates 3D models. Scans from 2 buildings were used for training and testing was conducted on the 3rd building. The meshes were pre-processed to compute space traversable by the robot, and a directed graph $\mathcal{G}_{x,\theta}$ was computed to represent the positions the robot could visit and the set of actions available at each node.

## 4. Mapping

![Mapper Architecture](https://i.imgur.com/T3w6f7h.png)

The mapping portion of their system amounts to latent representation, as it's fed directly into the planning module. It doesn't need to encode purely free space representations but instead functions as a general spatial memory that stores whatever's most useful for generating plans. The architecture follows.

At each time step $t$, we maintain a "cumulative estimate" of the free space $f_t$ in the coordinate frame of the robot. $f_t$ is represented as a multi-channel 2D feature map metrically representing a top-down map of the world. Each $f_t$ is estimated from current image $I_t$, the cumulative previous estimate $f_{t-1}$, and egomotion (between the last step and this step) $e_t$ using the following rule:   
$f_t=U(W(f_{t-1},e_t),f'_t)$, where:
* $f'_t=\phi(I_t)$.
* $W$ is a function that transforms the previous free space prediction $f_{t-1}$ according to the egomotion $e_t$.
* $\phi$ is a function that takes input current image $I_t$ and outputs an estimate of the free space based on the robot's current viewpoint. This estimate is denoted as $f'_t$.
* $U$ is a function that accumulates the free space predictions.

As a reminder, function $W$ translates the previous free space prediction to the current viewpoint using egomotion $e_t$. Given the ego-motion $e_t$, we compute a *backward flow field* $\rho(e_t)$ which maps each pixel in the current free space image $f_t$ to its location in the previous image $f_{t-1}$. $\rho$ can be computed using $e_t$, and $W$ then uses bi-linear sampling to apply the flow field to the free space estimate from the previous frame. It's this bi-linear sampling that allows back-propagation of gradients from $f_t$ to $f_{t-1}$, which allows end-to-end training. See [here](https://papers.nips.cc/paper/5854-spatial-transformer-networks.pdf).

Function $\phi$ is realized as a CNN. Because free space is always in the coordinate frame of the robot, the network only needs to output free space in the current coordinate. Intuitively, the network can use semantic cues (e.g. scene surfaces like floors or walls, or common furniture) to generate free space estimates; thus it can make predictions about unobserved spaces. The structure of the CNN is in Fig. 2: it uses a *convolutional encoder with residual connections* (see [here](https://arxiv.org/pdf/1512.03385.pdf)) that produces a representation of the scene in the 2D image space that's then transformed into the egocentric top-down view using fully connected layers. This is then up-sampled to obtain the belief update about the world for the current frame.

In addition to the current free space estimate $f'_t$, the model also produces a confidence $c'_t$. This estimate is warped by the warping function $W$ and accumulated over time into $c_t$, which allows the simplification of the update function (akin to an update gate in a GRU). The update function $U$ takes tuples $(f_{t-1},c_{t-1})$, and $(f'_t,c'_t)$ to produce $(f_t,c_t)$ as follows:   
$f_t=\frac{f_{t-1}c_{t-1}+f'_tc'_t}{c_{t-1}+c'_t}$ and $c_t=c_{t-1}+c'_t$. This function was chosen to keep the architecture simple, but it could be replaced with more expressive functions such as those in LSTMs.

## 5. Planning

Their planner is based on the *value iteration networks* proposed by [Tamar et al](https://papers.nips.cc/paper/6046-value-iteration-networks.pdf). These networks implement value iteration using alternating convolutions and channel-wise max pooling. Value iteration can be thought of as a generalization of Dijkstra's algorithm, where each node's value is iteratively recalculated using the max of its neighbors plus the value of the connecting transition. This works well in 2D grid worlds, where these operations can be implemented as 3x3 kernels following max-pooling over channels. Tamar showed that this reformulation can allow differentiable training of a planner on such a 2D grid world.

## To Look Up

* Bi-linear sampling and backward flow fields
* All links above here, these should be it:
* http://buildingparser.stanford.edu/images/3D_Semantic_Parsing.pdf
* https://papers.nips.cc/paper/5854-spatial-transformer-networks.pdf
* https://arxiv.org/pdf/1512.03385.pdf
* Up-convolutional layers
* https://papers.nips.cc/paper/6046-value-iteration-networks.pdf
* Value iteration

## Future Work Ideas

* Dynamic environments - What about dealing with pedestrians? Moving cars?
* Joint learning odometry/camera position in the environment
* Making use of floor plans or room signs etc. but not requiring them
* More expressive mapper update function

*- To be continued... -*   
Last revised 6/12/2019